In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
import glob
from PIL import Image



def threshold(img, threshold):
    img = img[:,:,0] #only necessary if image has rgb channels
    for n in range(0,img.shape[0]):
        for m in range(0,img.shape[1]):
            if img[n,m] > threshold:
                img[n,m] = 0
            else:
                img[n,m] = 1

    return img

def centroid(img):
    # Find the non-zero indices
    nonzero_indices = np.nonzero(img)

    # Calculate the centroid coordinates
    centroid_x = np.mean(nonzero_indices[1])
    centroid_y = np.mean(nonzero_indices[0])
    return centroid_x, centroid_y

def edge_detection_distances(img, centroid_x, centroid_y):
    # Calculate the gradient of the image
    gradient_x, gradient_y = np.gradient(img)

    # Find the edge pixels
    edge_pixels = np.sqrt(gradient_x**2 + gradient_y**2) > 0

    distance = np.zeros((img.shape[0], img.shape[1]))
    for n in range (0,img.shape[0]):
        for m in range (0,img.shape[1]):
            if edge_pixels[n,m] == True:
                distance[n,m] = np.sqrt((m - centroid_x)**2 + (n - centroid_y)**2)


    return distance


def convert_to_polar(img,distance, centroid_x, centroid_y):
    angles = np.array([])
    distances = np.array([])
    for n in range (0,img.shape[0]):
        for m in range (0,img.shape[1]):
            angle = np.arctan2(n - centroid_y, m - centroid_x)
            if distance[n,m] > 0:
                angles = np.append(angles, angle)
                distances = np.append(distances, distance[n,m])

    return angles, distances

def interpolate_polar(angles, distances):
    sorted_indices = np.argsort(angles)
    sorted_angles = angles[sorted_indices]
    sorted_distances = distances[sorted_indices]
    cs = interp1d(sorted_angles, sorted_distances)
    xs = np.linspace(sorted_angles[0], sorted_angles[-1], 1000)
    return xs, cs(xs)


                



def decompose_fft(data: list, threshold: float = 0.0):
    fft3 = np.fft.fft(data)
    x = np.arange(0, 2*np.pi, 2*np.pi / len(data))
    freqs = np.fft.fftfreq(len(x), 2*np.pi / len(x))
    recomb = np.zeros((len(x),))
    for i in range(len(fft3)):
        if abs(fft3[i]) / len(x) > threshold:
            sinewave = (
                1 
                / len(x) 
                * (
                    fft3[i].real 
                    * np.cos(freqs[i] * 2 * np.pi * x) 
                    - fft3[i].imag 
                    * np.sin(freqs[i] * 2 * np.pi * x)))
            recomb += sinewave
            plt.polar(x,recomb)
            plt.polar(x, sinewave)  # Plotting on a polar graph
    plt.show()

    plt.polar(x, recomb, label='Recombined')  # Plotting on a polar graph
    plt.polar(x, data, label='Original')  # Plotting on a polar graph
    plt.legend()  # Adding legend
    plt.show()
    return x, recomb
    

def master_particle_fft(img,threshold_fft, threshold_edge):
    img = threshold(img, threshold_edge)
    centroid_x, centroid_y = centroid(img)
    distance = edge_detection_distances(img, centroid_x, centroid_y)
    angles, distances_polar = convert_to_polar(img, distance, centroid_x, centroid_y)
    xs, cs = interpolate_polar(angles, distances_polar)

    return decompose_fft(cs, threshold_fft)

img = img = Image.open('B3P0X1.png')
img = np.array(img)
master_particle_fft(img, 2, 173)


In [ ]:
# put in a check to see if center is inside or outside of particle - Langdon
# figure out why there are a bunch of zeros at the beginning and end of the list - Donovan

def roc_dist(x, recomb, width):
    '''
    x:: array of angles
    recomeb:: array of radius values
    width:: width of the chord

    returns:: array of radii of curvature

    convert recomb and x from polar to cartesian coordinates;
    for a given angle, finds all of the points within the width of the chord;
    find the midpoint of the chord and froms the tangent line to the chord;
    calculates the point that is most equidistant from the arc;
    iterates one pixel around the edge
    '''
    radii = np.zeros((len(x),1))
    x_cartesian = recomb * np.cos(x)
    y_cartesian = recomb * np.sin(x)
    idx = np.where((x > width)&(x+width < 2*np.pi))
    for t in range (idx[0][0],idx[0][-1]):

        x_within_range = x[(x >= x[t] - width) & (x <= x[t] + width)]
        y_within_range = recomb[(x >= x[t] - width) & (x <= x[t] + width)]
        
        x_within_range_cartesian = y_within_range * np.cos(x_within_range)
        y_within_range_cartesian = y_within_range * np.sin(x_within_range)
        # print(t, y_within_range)

        a = np.array([x_within_range_cartesian[0],y_within_range_cartesian[0]])
        b = np.array([x_within_range_cartesian[-1],y_within_range_cartesian[-1]])

        d_init = np.linalg.norm(a-b)

        # Calculate the midpoint of the chord AB
        midpoint = (a + b) / 2

        # Calculate the slope of the chord AB
        slope_ab = (b[1] - a[1]) / (b[0] - a[0])

        # Calculate the slope of the perpendicular line
        slope_perpendicular = -1 / slope_ab

        # Calculate the y-intercept of the perpendicular line
        y_intercept = midpoint[1] - slope_perpendicular * midpoint[0]

        # Define a range of x-values for the line
        x_range = np.linspace(-1000, 1000, 10000)

        # Calculate the corresponding y-values using the equation of the perpendicular line
        y_range = slope_perpendicular * x_range + y_intercept





        # Calculate the distances from each point on x_within_range_cartesian, y_within_range_cartesian to each point on x_range, y_range
        distances = np.sqrt((x_within_range_cartesian[:, np.newaxis] - x_range)**2 + (y_within_range_cartesian[:, np.newaxis] - y_range)**2)

        # Calculate the average distance for each point on x_range, y_range
        average_distances = np.std(distances, axis=0)

        # Find the index of the point with the minimum average distance
        min_distance_index = np.argmin(average_distances)

        # Get the coordinates of the point with the minimum average distance
        equidistant_point = (x_range[min_distance_index], y_range[min_distance_index])

        center = np.mean(np.array([x_within_range_cartesian, y_within_range_cartesian]), axis=1)

        distance = np.sqrt((equidistant_point[0] - center[0])**2 + (equidistant_point[1] - center[1])**2)
        
        radii[t] = distance

    return radii

radii = roc_dist(x, recomb, width)

plt.hist(radii[np.nonzero(radii)], bins=500)
        

    
    